In [84]:
from google.colab import drive
import os, json, time, math
import numpy as np

if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted")

DATA_DIR = "/content/drive/MyDrive/tokenized_subset"
BIN_DIR  = os.path.join(DATA_DIR, "bin")

TRAIN_BIN = os.path.join(BIN_DIR, "train.bin")
VAL_BIN   = os.path.join(BIN_DIR, "val.bin")
VOCAB_JSON = os.path.join(DATA_DIR, "vocab.json")
TEST_BIN = os.path.join(BIN_DIR, "test.bin")
assert os.path.exists(TRAIN_BIN), f"Missing {TRAIN_BIN}"
assert os.path.exists(VAL_BIN), f"Missing {VAL_BIN}"
assert os.path.exists(VOCAB_JSON), f"Missing {VOCAB_JSON}"

print("Found:", TRAIN_BIN, VAL_BIN, VOCAB_JSON)


Google Drive already mounted
Found: /content/drive/MyDrive/tokenized_subset/bin/train.bin /content/drive/MyDrive/tokenized_subset/bin/val.bin /content/drive/MyDrive/tokenized_subset/vocab.json


In [26]:
with open(VOCAB_JSON, "r") as f:
    vocab = json.load(f)

if all(isinstance(v, int) for v in vocab.values()):
    token_to_id = vocab
    id_to_token = {i: t for t, i in token_to_id.items()}
else:

    id_to_token = {int(k): v for k, v in vocab.items()}
    token_to_id = {v: k for k, v in id_to_token.items()}

vocab_size = len(id_to_token)
print("Vocab size:", vocab_size)


Vocab size: 1734


In [27]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

class BinDataset:
    def __init__(self, path, block_size):
        self.data = np.memmap(path, dtype=np.uint32, mode='r')
        self.block_size = block_size

    def get_batch(self, batch_size, device):
        ix = torch.randint(len(self.data) - self.block_size - 1, (batch_size,))
        x = torch.stack([
            torch.from_numpy(self.data[i:i+self.block_size].astype(np.int64))
            for i in ix
        ])
        y = torch.stack([
            torch.from_numpy(self.data[i+1:i+self.block_size+1].astype(np.int64))
            for i in ix
        ])
        return x.to(device), y.to(device)

block_size = 256
train_data = BinDataset(TRAIN_BIN, block_size)
val_data   = BinDataset(VAL_BIN, block_size)

total_train_tokens = len(train_data.data)
print("Total train tokens:", total_train_tokens)


Using device: cuda
Total train tokens: 140459152


In [28]:
import torch.nn as nn
import torch.nn.functional as F

class CausalSelfAttention(nn.Module):
    def __init__(self, n_embd, n_head, block_size, attn_dropout=0.1, resid_dropout=0.1):
        super().__init__()
        assert n_embd % n_head == 0
        self.n_head = n_head
        self.head_dim = n_embd // n_head

        self.key   = nn.Linear(n_embd, n_embd, bias=False)
        self.query = nn.Linear(n_embd, n_embd, bias=False)
        self.value = nn.Linear(n_embd, n_embd, bias=False)
        self.proj  = nn.Linear(n_embd, n_embd, bias=False)

        self.attn_drop = nn.Dropout(attn_dropout)
        self.resid_drop = nn.Dropout(resid_dropout)

        self.register_buffer("mask", torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.size()

        k = self.key(x).view(B, T, self.n_head, self.head_dim).transpose(1,2)
        q = self.query(x).view(B, T, self.n_head, self.head_dim).transpose(1,2)
        v = self.value(x).view(B, T, self.n_head, self.head_dim).transpose(1,2)

        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        att = att.masked_fill(self.mask[:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)

        y = att @ v
        y = y.transpose(1,2).contiguous().view(B, T, C)
        y = self.proj(y)
        y = self.resid_drop(y)
        return y

class Block(nn.Module):
    def __init__(self, n_embd, n_head, block_size, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.attn = CausalSelfAttention(n_embd, n_head, block_size, attn_dropout=dropout, resid_dropout=dropout)
        self.ln2 = nn.LayerNorm(n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.GELU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

class TransformerLM(nn.Module):
    def __init__(self, vocab_size, n_layer, n_head, n_embd, block_size, dropout=0.1):
        super().__init__()
        self.block_size = block_size
        self.tok_emb = nn.Embedding(vocab_size, n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, block_size, n_embd))
        self.drop = nn.Dropout(dropout)
        self.blocks = nn.ModuleList([Block(n_embd, n_head, block_size, dropout=dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.head = nn.Linear(n_embd, vocab_size, bias=False)

        nn.init.normal_(self.pos_emb, mean=0.0, std=0.02)

    def forward(self, x, targets=None):
        B, T = x.size()
        assert T <= self.block_size
        x = self.tok_emb(x) + self.pos_emb[:, :T]
        x = self.drop(x)
        for blk in self.blocks:
            x = blk(x)
        x = self.ln_f(x)
        logits = self.head(x)

        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

def count_params_M(model):
    return sum(p.numel() for p in model.parameters()) / 1e6

@torch.no_grad()
def estimate_val_loss(model, data, batch_size, iters=200):
    model.eval()
    losses = []
    for _ in range(iters):
        xb, yb = data.get_batch(batch_size, device)
        _, loss = model(xb, yb)
        losses.append(loss.item())
    model.train()
    return sum(losses) / len(losses)


In [29]:
from pathlib import Path

def save_ckpt(path, model, opt, step, tokens_seen, best_val):
    ckpt = {
        "model": model.state_dict(),
        "opt": opt.state_dict(),
        "step": step,
        "tokens_seen": tokens_seen,
        "best_val": best_val,
    }
    torch.save(ckpt, path)

def load_ckpt(path, model, opt=None):
    ckpt = torch.load(path, map_location="cpu")
    model.load_state_dict(ckpt["model"])
    if opt is not None and "opt" in ckpt:
        opt.load_state_dict(ckpt["opt"])
    return ckpt

def train_best_model_time_budget(
    cfg,
    train_data,
    val_data,
    time_budget_min=120,
    batch_size=8,
    grad_accum_steps=4,
    lr=3e-4,
    min_lr=3e-5,
    weight_decay=0.1,
    grad_clip=1.0,
    warmup_frac=0.02,
    eval_every=5000,
    val_iters=200,
    ckpt_dir="/content/drive/MyDrive/best_model_part4",
    ckpt_name="best_transformer.pt",
    resume=True,
    use_compile=True
):
    os.makedirs(ckpt_dir, exist_ok=True)
    ckpt_path = os.path.join(ckpt_dir, ckpt_name)
    last_path = os.path.join(ckpt_dir, "last.pt")

    model = TransformerLM(**cfg).to(device)
    print("Params (M):", count_params_M(model))

    opt = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.95), weight_decay=weight_decay)


    scaler = torch.amp.GradScaler('cuda', enabled=(device == "cuda"))


    step = 0
    tokens_seen = 0
    best_val = float("inf")
    if resume and os.path.exists(last_path):
        ck = load_ckpt(last_path, model, opt)
        step = ck.get("step", 0)
        tokens_seen = ck.get("tokens_seen", 0)
        best_val = ck.get("best_val", best_val)
        print(f"Resumed from {last_path} | step={step} tokens_seen={tokens_seen} best_val={best_val:.4f}")

    if use_compile:
        try:
            model = torch.compile(model)
            print("torch.compile enabled ✅")
        except Exception as e:
            print("torch.compile failed (continuing without):", e)


    warmup_steps = max(10, int(warmup_frac * 200000))
    def get_lr(s):
        if s < warmup_steps:
            return lr * (s / warmup_steps)

        progress = (s - warmup_steps) / max(1, (200000 - warmup_steps))
        progress = min(1.0, progress)
        return max(min_lr, lr * 0.5 * (1.0 + math.cos(math.pi * progress)))

    tokens_per_step = batch_size * cfg["block_size"] * grad_accum_steps
    print("Tokens/optimizer-step:", tokens_per_step)

    torch.cuda.reset_peak_memory_stats()
    t0 = time.time()
    deadline = t0 + time_budget_min * 60

    model.train()
    loss_ema = None
    ema_beta = 0.95

    while time.time() < deadline:
        lr_now = get_lr(step)
        for pg in opt.param_groups:
            pg["lr"] = lr_now

        opt.zero_grad(set_to_none=True)

        total_loss = 0.0
        for micro in range(grad_accum_steps):
            xb, yb = train_data.get_batch(batch_size, device)
            with torch.autocast(device_type="cuda", dtype=torch.float16, enabled=(device=="cuda")):
                _, loss = model(xb, yb)
                loss = loss / grad_accum_steps
            scaler.scale(loss).backward()
            total_loss += loss.item()

        scaler.unscale_(opt)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        scaler.step(opt)
        scaler.update()

        tokens_seen += tokens_per_step
        loss_val = total_loss
        loss_ema = loss_val if loss_ema is None else (ema_beta * loss_ema + (1-ema_beta) * loss_val)

        if step % 200 == 0:
            elapsed = (time.time() - t0) / 60
            peak = torch.cuda.max_memory_allocated() / 1024**2 if device=="cuda" else 0
            print(f"step {step} | lr {lr_now:.2e} | loss_ema {loss_ema:.4f} | tokens {tokens_seen:,} | {elapsed:.1f} min | peak {peak:.0f} MB")


        if step > 0 and step % eval_every == 0:
            val_loss = estimate_val_loss(model, val_data, batch_size=batch_size, iters=val_iters)
            print(f"✅ eval @ step {step}: val_loss={val_loss:.4f} (best={best_val:.4f})")

            save_ckpt(last_path, model, opt, step, tokens_seen, best_val)

            if val_loss < best_val:
                best_val = val_loss
                save_ckpt(ckpt_path, model, opt, step, tokens_seen, best_val)
                print(f"🏆 New best saved to: {ckpt_path}")

        step += 1


    save_ckpt(last_path, model, opt, step, tokens_seen, best_val)
    print("Finished. Saved last checkpoint to:", last_path)

    peak = torch.cuda.max_memory_allocated() / 1024**2 if device=="cuda" else 0
    return model, {"step": step, "tokens_seen": tokens_seen, "best_val": best_val, "peak_mem_MB": peak, "best_ckpt": ckpt_path, "last_ckpt": last_path}


In [30]:
best_cfg = dict(
    vocab_size=vocab_size,
    n_layer=20,
    n_head=20,
    n_embd=640,
    block_size=256,
    dropout=0.1,
)

time_budget_min = 120

model, stats = train_best_model_time_budget(
    best_cfg,
    train_data,
    val_data,
    time_budget_min=time_budget_min,
    batch_size=8,
    grad_accum_steps=4,
    lr=3e-4,
    min_lr=3e-5,
    weight_decay=0.1,
    eval_every=2000,
    val_iters=200,
    ckpt_dir="/content/drive/MyDrive/best_model_part4",
    ckpt_name="best_transformer.pt",
    resume=True,
    use_compile=True
)

print("Training stats:", stats)


Params (M): 100.80384
torch.compile enabled ✅
Tokens/optimizer-step: 8192
step 0 | lr 0.00e+00 | loss_ema 7.7350 | tokens 8,192 | 0.8 min | peak 7743 MB
step 200 | lr 1.50e-05 | loss_ema 3.3434 | tokens 1,646,592 | 1.3 min | peak 8522 MB
step 400 | lr 3.00e-05 | loss_ema 2.6267 | tokens 3,284,992 | 1.8 min | peak 8522 MB
step 600 | lr 4.50e-05 | loss_ema 2.4015 | tokens 4,923,392 | 2.3 min | peak 8522 MB
step 800 | lr 6.00e-05 | loss_ema 2.2934 | tokens 6,561,792 | 2.8 min | peak 8522 MB
step 1000 | lr 7.50e-05 | loss_ema 2.1609 | tokens 8,200,192 | 3.3 min | peak 8522 MB
step 1200 | lr 9.00e-05 | loss_ema 2.0473 | tokens 9,838,592 | 3.8 min | peak 8522 MB
step 1400 | lr 1.05e-04 | loss_ema 1.9509 | tokens 11,476,992 | 4.3 min | peak 8522 MB
step 1600 | lr 1.20e-04 | loss_ema 1.8961 | tokens 13,115,392 | 4.8 min | peak 8522 MB
step 1800 | lr 1.35e-04 | loss_ema 1.8407 | tokens 14,753,792 | 5.3 min | peak 8522 MB
step 2000 | lr 1.50e-04 | loss_ema 1.8254 | tokens 16,392,192 | 5.8 min | 

In [73]:
import torch

def encode_notes(notes):
    """
    notes: list of ABC note tokens, e.g. ["C", "E", "G"]
    """
    return torch.tensor(
        [[token_to_id.get(n, token_to_id["<UNK>"]) for n in notes]],
        dtype=torch.long
    ).to(device)


def decode_notes(ids):
    """
    ids: list of token IDs
    """
    return " ".join(id_to_token[i] for i in ids if i in id_to_token)


In [74]:
@torch.no_grad()
def generate_notes(
    model,
    start_ids,
    max_new_tokens=600,
    temperature=0.9,
    top_k=40
):
    model.eval()
    out = start_ids.clone()

    for _ in range(max_new_tokens):
        idx = out[:, -model.block_size:]
        logits, _ = model(idx)

        logits = logits[:, -1, :] / temperature

        if top_k is not None:
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float("inf")

        probs = torch.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, 1)

        out = torch.cat([out, next_id], dim=1)

    return out


In [75]:
def postprocess_notes(note_str, max_consecutive_rests=2):
    """
    Limits excessive 'z' rest tokens that cause long silence.
    """
    tokens = note_str.split()
    cleaned = []
    rest_count = 0

    for t in tokens:
        if t.startswith("z"):
            rest_count += 1
            if rest_count <= max_consecutive_rests:
                cleaned.append("z")
        else:
            rest_count = 0
            cleaned.append(t)

    return " ".join(cleaned)


In [76]:
ABC_HEADER = """X:1
T:Generated Sample
M:4/4
L:1/8
Q:1/4=120
K:C
"""


In [77]:
import os

ABC_OUT = "/content/drive/MyDrive/generated_abc"
os.makedirs(ABC_OUT, exist_ok=True)


start_uncond = encode_notes(["C"])

for i in range(5):
    out = generate_notes(
        model,
        start_uncond,
        max_new_tokens=600,
        temperature=0.9,
        top_k=40
    )

    notes = decode_notes(out[0].cpu().tolist())
    body = postprocess_notes(notes)

    abc = ABC_HEADER + body + "\n"
    with open(f"{ABC_OUT}/unconditional_{i}.abc", "w") as f:
        f.write(abc)

print("✅ 5 unconditional samples generated")



prefix = encode_notes(["C", "E", "G"])

for i in range(5):
    out = generate_notes(
        model,
        prefix,
        max_new_tokens=600,
        temperature=0.9,
        top_k=40
    )

    notes = decode_notes(out[0].cpu().tolist())
    body = postprocess_notes(notes)

    abc = ABC_HEADER + body + "\n"
    with open(f"{ABC_OUT}/conditional_{i}.abc", "w") as f:
        f.write(abc)

print("✅ 5 conditional samples generated")


✅ 5 unconditional samples generated
✅ 5 conditional samples generated


In [78]:
from music21 import converter, stream

MIDI_OUT = "/content/drive/MyDrive/generated_midi"
os.makedirs(MIDI_OUT, exist_ok=True)

converted, failed = 0, 0

for file in os.listdir(ABC_OUT):
    if not file.endswith(".abc"):
        continue

    abc_path = os.path.join(ABC_OUT, file)
    midi_path = os.path.join(MIDI_OUT, file.replace(".abc", ".mid"))

    try:
        score = converter.parse(abc_path)


        flat = score.flatten()
        s = stream.Score()
        s.insert(0, flat)

        s.write("midi", midi_path)
        converted += 1
    except Exception as e:
        print("❌ Failed:", file, e)
        failed += 1

print(f"\n✅ MIDI conversion complete")
print(f"Converted: {converted}")
print(f"Failed   : {failed}")



✅ MIDI conversion complete
Converted: 10
Failed   : 0


In [85]:
block_size = 256

test_data = BinDataset(TEST_BIN, block_size)

print("Total test tokens:", len(test_data.data))


Total test tokens: 1426554


In [86]:
import math
import torch

@torch.no_grad()
def evaluate_perplexity(model, test_data, batch_size=8, iters=500):
    model.eval()
    losses = []

    for _ in range(iters):
        xb, yb = test_data.get_batch(batch_size, device)
        _, loss = model(xb, yb)
        losses.append(loss.item())

    avg_loss = sum(losses) / len(losses)
    perplexity = math.exp(avg_loss)
    return avg_loss, perplexity


test_loss, test_ppl = evaluate_perplexity(
    model,
    test_data,
    batch_size=8,
    iters=500
)

print(f"Final Test Loss: {test_loss:.4f}")
print(f"Final Test Perplexity: {test_ppl:.2f}")


Final Test Loss: 2.2192
Final Test Perplexity: 9.20


In [87]:
from music21 import converter
import tempfile
import os

def is_valid_abc(abc_text, min_tokens=20):
    tokens = abc_text.strip().split()
    if len(tokens) < min_tokens:
        return False

    try:
        with tempfile.NamedTemporaryFile("w", suffix=".abc", delete=False) as f:
            f.write(abc_text)
            temp_path = f.name

        converter.parse(temp_path)
        return True
    except Exception:
        return False
    finally:
        if os.path.exists(temp_path):
            os.remove(temp_path)


In [90]:
ABC_OUT = "/content/drive/MyDrive/generated_abc"

valid = 0
total = 0

for file in os.listdir(ABC_OUT):
    if not file.endswith(".abc"):
        continue

    total += 1
    with open(os.path.join(ABC_OUT, file)) as f:
        abc = f.read()

    if is_valid_abc(abc):
        valid += 1

valid_pct = 100 * valid / total

print(f"Syntactically valid ABC files: {valid}/{total}")
print(f"Percentage valid: {valid_pct:.2f}%")


Syntactically valid ABC files: 10/10
Percentage valid: 100.00%


In [91]:
from music21 import converter

MIDI_OUT = "/content/drive/MyDrive/generated_midi"
os.makedirs(MIDI_OUT, exist_ok=True)

converted = 0
failed = 0

for file in os.listdir(ABC_OUT):
    if not file.endswith(".abc"):
        continue

    abc_path = os.path.join(ABC_OUT, file)
    midi_path = os.path.join(MIDI_OUT, file.replace(".abc", ".mid"))

    try:
        score = converter.parse(abc_path)
        score.write("midi", midi_path)
        converted += 1
    except Exception:
        failed += 1

total = converted + failed
midi_pct = 100 * converted / total

print(f"MIDI conversion success: {converted}/{total}")
print(f"Conversion success rate: {midi_pct:.2f}%")


MIDI conversion success: 10/10
Conversion success rate: 100.00%
